# Generate Parallel Dataset

The clean dataset is a modernization of the CATSS Database, presented with minimal changes, in UTF8, exported as JSON.

In [1]:
import sys
import regex 
import collections
import pandas as pd
from pathlib import Path
from greekutils import beta2unicode # do: pip install greek-utils==0.2
from pprint import pprint

sys.path.append('../')
import regex_patterns as repatts

data = Path('../source/patched')

In [2]:
# CCAT transcription to UTF8
# Greek to be handled by greekutils

# Hebrew
trans2utf8 = {
    ')': 'א',
    'B': 'ב',
    'G': 'ג',
    'D': 'ד',
    'H': 'ה',
    'W': 'ו',
    'Z': 'ז',
    'X': 'ח',
    '+': 'ט',
    'Y': 'י',
    'K': 'כ',
    'L': 'ל',
    'M': 'מ',
    'N': 'נ',
    'S': 'ס',
    '(': 'ע',
    'P': 'פ',
    'C': 'צ',
    'Q': 'ק',
    'R': 'ר',
    '&': 'שׂ',
    '$': 'שׁ',
    'T': 'ת',
    '-': '־',
    '\\': '',
    ' ': ' ',
}

hfletter = r'{}(?=\s|$)'
final_heb = (
    (hfletter.format('\u05DE'), 'ם'),
    (hfletter.format('\u05DB'), 'ך'),
    (hfletter.format('\u05E0'), 'ן'),
    (hfletter.format('\u05E4'), 'ף'),
    (hfletter.format('\u05E6'), 'ץ'),
)
final_heb = [(regex.compile(patt), repl) for patt,repl in final_heb]

def sub_final_hb(string):
    """Substitute final letters in Hebrew"""
    for patt, repl in final_heb:
        string = patt.sub(repl, string)
    return string

def utf8_hebrew(string):
    """Convert transcribed Hebrew to UTF8
    
    NB: does not provide final letters (e.g. ם).
    """
    utf8_string = ''
    for c in string:
        utf8_string += trans2utf8.get(c, '')
    utf8_string = sub_final_hb(utf8_string)
    return utf8_string

In [3]:
beta2unicode.convert('SUNAGWGH\\N')

'συναγωγὴν'

In [4]:
utf8_hebrew('HYTH $M B\BYT-LXMM')

'היתה שׁם בבית־לחמם'

## Process the files

We process the CATSS database files into JSONs. 

The datastructure is illustrated below:

**NB: hypothetical data for illustration purposes**

```
[ # list of verses
    
    [ # list data for a verse
    
        'Gen 1:9',     # verse reference
        
        [ # three-list of text columns
            
            [ # Hebrew column A
                ('מקום', {'retcon'}), # text entry + text critical notes
            ],
            
            [ # Hebrew column B
                ('מקוה', {'?'}), # text entry + text critical notes
            ], 
            
            [ # Greek column
                ('συναγωγὴν', {''}), # text entry + text critical notes
            ],   
        ],
    ],
]
```

Note that some captured contexts may be empty. In this case, the notes are added to empty strings.

Each list consists of a single verse, headed by a reference string. The 
second element in the list is a three-list of two-tuples.

Each two-tuple represents a column in the database, and they consist 
of `(text, text-critical notes)`. Hebrew column B contains retroverted
readings and it is frequently empty.

Each column can contain multiple text entries, for cases where there are 
separate notes per column. For instance, the database might contain 
words wrapped in curly brackets `{}` with notations that are separate
from another word that is not contained in them. Thus, each column
can contain more than 1 entry.

## Parsing Strategy

Each line contains 2-3 columns of data. Those columns contain original language text and markup.

We will draw a strong distinction between text and markup, and seek to separate the two. 

To do this, we first run a pattern search for markup patterns. The markup patterns divide into
several subsets, depending on their behavior:

* substitutions - represents text elements that have been transported elsewhere (contains no text elements)
* context-based - markup that applies within a context of a stretch of text
* capturers - markup that captures text within its vicinity based on capture groups

If all markup patterns fail to match a string, the algorithm will then run the regex patterns for matching 
original language text to recognize it as text.

A "context" is an important concept for the parser. The highest level context is a given column
(for instance, Hebrew column A). Contexts can then be split into smaller pieces based on markup
capture groups. For instance, the following markup: `{...MQVH <1:23>}` captures everything in
between the brackets. Everything in the brackets then becomes a new, smaller context. The pattern
matches are run again, recursively, against this context to recognize the text and the additional
markup. That markup, if it is of the context-based type, will then only be applied to this word.

In [5]:
# compile the patterns for matching

# original language text
hchars = repatts.hchars
gchars = repatts.gchars
hb_patt = regex.compile(f' *[{hchars}]+ *')
grk_patt = regex.compile(f' *[{gchars}]+ *')

# markup text
comp_patt = lambda pattern: (regex.compile(pattern[0]),) + pattern[1:]
common_tc_patts = [comp_patt(p) for p in repatts.common_tc]
hb_tc_patts = common_tc_patts + [comp_patt(p) for p in repatts.heb_tc]
gk_tc_patts = common_tc_patts + [comp_patt(p) for p in repatts.greek_tc]

In [13]:
def normalize_element(element):
    return element.strip()

def parse_context(context, markup_patts, text_patt, position=0, column_list=[], markups=set(), ident='', debug=[]):
    """Parse a context of text and markup in structured JSON."""
    
    elements = []
    
    def report(*messages):
        debug.extend(ident+m for m in messages)
    
    report(f'analyzing context: {context}')
    
    while context and (position < len(context)):
    
        matched = False # track matches in the loop
        
        for patt, kind, tag, desc, indices in markup_patts:
            
            # process markup
            if match := patt.match(context, position):
                    
                # run any optional formats on tag
                tag = tag.format(**{k:(match.groups()[i] or '') for k, i in indices.items()})
                    
                # tag markup within the context
                if kind == 'con':
                    report(f'  markup pattern match @ {position}: {patt.pattern}', f'    match: {match.group(0)}')
                    markups.add(tag)

                # run parser recursively for captured sub-contexts
                elif kind == 'cap':
                    report(f'  markup pattern match @ {position}: {patt.pattern}')
                    subcontext = match.groups()[indices['txt']]
                    elements.extend(
                            parse_context(
                                subcontext, markup_patts, text_patt, markups={tag}, column_list=[],
                                ident=ident+'    ', debug=debug,
                            )
                    )

                # deal with substitution markups
                elif kind == 'sub':
                    elements.append(('', {tag}))
                    
                else:
                    raise Exception(f'PATTERN ERROR for {patt}: NO KIND')

                # advance the position
                position = match.end()
                matched = True
                break
                
        if not matched:

            # process original language text
            if match := text_patt.match(context, position):
                elements.append((match.group(), set()))
                report(f'\ttext match @ {position}: {match.group(0)}')
                position = match.end()

            # no match found, raise a syntax error
            elif position < len(context):
                error = f'SYNTAX ERROR AT POSITION {position} `{context[position]}` i.e. `{context[position-1:position+2]}`|{context}'
                raise Exception(error)
      
    # we're done
    # apply contextual markup to all elements
    # and return the goods
    if elements:
        for element, markup_set in elements:
            markup_set |= markups
            column_list.append((normalize_element(element), markup_set))
    elif markups:
        column_list.append(('', markups))
        
    # recursion depth limit
    if len(column_list) >= 100:
        error = 'RECURSION DEPTH LIMIT REACHED!'
        report('\n'.join('\t'+c for c in column_list))
        raise Exception(error)
        
    return column_list

In [14]:
test_context = "K/DMWT/NW {d} {...MLK %p}"
debug = []
parse_context(test_context, hb_tc_patts, hb_patt, column_list=[], debug=debug)

[('K/DMWT/NW', {'.doub'}), ('MLK', {'doub.', 'prep', 'trans'})]

In [15]:
print('\n'.join(debug))

analyzing context: K/DMWT/NW {d} {...MLK %p}
  markup pattern match @ 0: ({.*?}|[^\s]+) *(?=\??\{d\})
    analyzing context: K/DMWT/NW
    	text match @ 0: K/DMWT/NW
  markup pattern match @ 10: =?\{d\}\??\s*({.*?}|[^\s]*) *
    analyzing context: {...MLK %p}
      markup pattern match @ 0: {\.\.\.(.+?)} *
        analyzing context: MLK %p
        	text match @ 0: MLK 
          markup pattern match @ 4: =?%p([-+])? *
            match: %p


<hr>

In [16]:
# -- regex patterns --
continued_column = regex.compile(r'[^\s]+.*#\s*$') # '#' at end of col preceded by some non-space char
content = regex.compile(r'.*[^\s].*') # string has some non-space char (content)

def line_is_continued(col1, col2):
    """Return boolean whether any column in a line is continued in next line"""
    if continued_column.match(col1) or continued_column.match(col2):
        return True
    else:
        return False

def is_dataline(line):
    """Return boolean on whether a line contains data content"""
    return all([
        content.match(line), 
        not repatts.ref_string.match(line)
    ])

def get_continued_columns(lines, counter):
    """Recursively retrieve data-lines continued on next line (marked with #).
    
    The function recursively retrieves subsequent lines if a starting line
    is marked with a continuation marker (#). Each line that is retrieved
    must be split into its columns, and those columns in turn must be 
    checked for continuation markers. This is done recursively until there
    is no continuation marker found. The function retrieves the lines using 
    the current index position; it advances the index by adding 1 each time. 
    It yields all additional columns it finds as 2-tuples.
    """
    line = lines[counter]
    if is_dataline(line):
        heb_col, grk_col = line.split('\t')
        if line_is_continued(heb_col, grk_col):
            counter += 1
            next_cols = lines[counter].split('\t')
            yield next_cols
            yield from get_continued_columns(lines, counter) # recursive call here

def transcribe_hebrew(string):
    """Transcribe a string of Hebrew column text from the parallel database."""
    pass

def transcribe_greek(string):
    """Transcribe a string of Greek column text from the parallel database."""
    pass

def clean_text(string, lang='hebrew'):
    """Clean string of text from the parallel database."""
    pass

# finalized parallel data goes here
para_data = []
parsed = collections.defaultdict(list)
errors = []
debugs = []

# process files
print('beginning analysis of books\n')
for file in sorted(data.glob('*.par')):
    
    if file.name == '02.Exodus.par':
        print('DONE')
        print(f'\tn-parsed: {len(parsed)}')
        print(f'\tn-errors: {len(errors)}')
        break
    
    print(f'parsing {file.name}...')
    
    # read the file
    lines = file.read_text().split('\n')
    
    book_data = [file.stem]
    verse_data = []
    position = 0
    
    while position < len(lines):
    
        line = lines[position]
    
        # detect a new verse at verse reference string
        if repatts.ref_string.match(line):
            
            # store last verse, make space for new one, store new one
            if verse_data:
                book_data.append(verse_data)
                verse_data = []
            verse_data.append(line)
        
        elif line:
            
            # for debugging
            show_tuple = (file.name, position, verse_data[0], line)
            
            # extract the two columns
            heb_col, grk_col = line.split('\t')
            
            # NB: that for Sirach the Hebrew columns can sometimes
            # be split several ways since there are numerous Hebrew 
            # sources, deriving from various manuscripts
            # the sources are indicated by a following number;
            # thus, it may be possible to split along stand-alone integers
            # to divide up the text
            
            # seperate heb col a and b (optional)
            if '=' in heb_col:
                heb_colA, heb_colB = heb_col.split('=', 1)
            else:
                heb_colA = heb_col
                heb_colB = ''
            
            # collect parts of the columns continued on next line(s) in doc
            # this is done recursively to ensure all lines are retrieved
            cont_cols = list(get_continued_columns(lines, position))
            for hb_cc, gk_cc in cont_cols:
                position += 1
                heb_col += hb_cc
                grk_col += gk_cc
 
            # columns are now ready for the parser
            # feed into the parser, and if there is a problem
            # record it and move on
            grammars = [
                (heb_colA, hb_tc_patts, hb_patt), 
                (heb_colB, hb_tc_patts, hb_patt),
                (grk_col, gk_tc_patts, grk_patt),
            ]
            
            good = True
            debug = [file.name, str(position), f'line: {line}', '-'*30]
            column_parsings = []
            for context, markup_patts, text_patt in grammars:
                try:
                    this_parse = parse_context(
                        context, 
                        markup_patts,
                        text_patt,
                        debug=debug,
                        column_list=[],
                        markups=set(),
                    )
                    column_parsings.append(this_parse)
                except:
                    einfo = ' '.join(str(e) for e in list(sys.exc_info())[:2])
                    debug.append(einfo)
                    errors.append(debug)
                    good = False
                    break
              
            if good:
                parsed[f'{file.name}.{position}'] = column_parsings
                debugs.append(debug)
                verse_data.extend(column_parsings)
              
        # it's an empty line; move on
        else:
            pass
            
        position += 1
        
    para_data.append(book_data)
    print(f'\tbook successfully parsed.')
    
def show_errors(errors):
    for error in errors:
        print('\n'.join(error))
        print()

beginning analysis of books

parsing 01.Genesis.par...
	book successfully parsed.
DONE
	n-parsed: 20421
	n-errors: 70


In [17]:
# Regex Testing Cell

test_re = regex.compile("(^| +)\?+( +|$)")
test_str = "? ---"
position = 0
match = test_re.match(test_str, position)
match

<regex.Match object; span=(0, 2), match='? '>

In [18]:
parsed['01.Genesis.par.719']

[[('W/L/$MR/H', set())],
 [],
 [('KAI\\', set()), ('FULA/SSEIN', set()), ('AU)TO\\N', {'distr'})]]

In [19]:
show_errors(errors[:5])

01.Genesis.par
1442
line: N( =@?)NX .(x	STE/NWN
------------------------------
analyzing context: N( 
	text match @ 0: N( 
analyzing context: @?)NX .(x
  markup pattern match @ 0: =?@([)BGDHWZX+YKLMNS(PCQR&$T/ ?,]*)
    analyzing context: ?)NX 
      markup pattern match @ 0: (^| +)\?+([^\s?]+ *)
        analyzing context: )NX 
        	text match @ 0: )NX 
<class 'Exception'> SYNTAX ERROR AT POSITION 6 `.` i.e. ` .(`|@?)NX .(x

01.Genesis.par
1466
line: N( =@?)NX .(x	STE/NWN
------------------------------
analyzing context: N( 
	text match @ 0: N( 
analyzing context: @?)NX .(x
  markup pattern match @ 0: =?@([)BGDHWZX+YKLMNS(PCQR&$T/ ?,]*)
    analyzing context: ?)NX 
      markup pattern match @ 0: (^| +)\?+([^\s?]+ *)
        analyzing context: )NX 
        	text match @ 0: )NX 
<class 'Exception'> SYNTAX ERROR AT POSITION 6 `.` i.e. ` .(`|@?)NX .(x

01.Genesis.par
1578
line: --+ =?HW) HYH <4.20>	{...KAI\ H)=N}?
------------------------------
analyzing context: --+ 
  markup pattern

## Testing the Line-merger

Here is a prototype for collecting the next lines recursively.

In [35]:
# test = Path('../source/patched/17.1Esdras.par').read_text().split('\n')

# lines = test[3067]

# lines

In [32]:

# i = 0
# while i < len(lines):
    
#     line = lines[i]
    
#     if not line or ref_string.match(line):
#         i += 1
#         continue
    
#     print(line)
#     heb_col, grk_col = line.split('\t')
#     cont_lines = list(get_continued_columns(lines, i))
#     #print(cont_lines)
#     for hb_cc, gk_cc in cont_lines:
#         i += 1 # advance position in doc
#         heb_col += hb_cc
#         grk_col += gk_cc

#     print('\theb:', heb_col)
#     print('\tgrk:', grk_col)
#     print()
            
#     i += 1

In [33]:
# cont_column = re.compile(r'[^\s]+.*#\s*$')

# for line in lines:
#     for col in line.split('\t'):
#         if cont_column.match(col):
#             print(col, 'match')

In [34]:
# line_is_continued(*'BR)Y W/B/$(RYM =:BR)WM$(RYM #\tBAROUMSEWRIM {t}'.split('\t'))